In [59]:
#!/usr/bin/env python3

import os
import sys
import logging
from datetime import datetime
import time
import requests

import geopandas as gpd
from shapely.geometry import shape
from shapely.geometry import Polygon
from shapely.geometry import Point
from shapely.geometry import LineString
from shapely.geometry import MultiLineString
from shapely import wkt
from shapely.geometry import JOIN_STYLE
import matplotlib.pyplot as plt 

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

export_raw = False

# Configuration
API_URL = "https://deepstatemap.live/api/history/last"
OUTPUT_DIR = "data"
OUTPUT_FILENAME = f"testfile_deepstatemap_data_{datetime.now().strftime('%Y%m%d')}.geojson"
MAX_RETRIES = 3
RETRY_DELAY = 5  # seconds


def make_api_request():
    """Make a request to the API and return the JSON response."""
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows Phone 10.0; Android 6.0.1; Microsoft; RM-1152) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Mobile Safari/537.36 Edge/15.15254"
    }
    
    for attempt in range(MAX_RETRIES):
        try:
            response = requests.get(API_URL, headers=headers, timeout=10)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            logger.warning(f"API request failed (attempt {attempt + 1}/{MAX_RETRIES}): {e}")
            if attempt < MAX_RETRIES - 1:
                logger.info(f"Retrying in {RETRY_DELAY} seconds...")
                time.sleep(RETRY_DELAY)
            else:
                logger.error("All API request attempts failed.")
                sys.exit(1)


def process_data(data):
    """Process the API response data."""

    geo_list = []
    for f in data['map']['features']:
        geom = f['geometry']
        name = f['properties']['name']
        new_feature = {
            "name":name,
            "geometry":wkt.loads(wkt.dumps(shape(geom), output_dimension=2))
            }

        geo_list.append(new_feature)
        
    # Split the name by '///' and take the first part
    def extract_first_part(name, part=0):
        first_part = name.split('///')[part].strip()
        return first_part
    
    for item in geo_list:
        item['name'] = extract_first_part(item['name'], part=1)
    
    return geo_list


def create_geodataframe(geo_list):
    """Create a GeoDataFrame from the processed data."""
    raw_gdf = gpd.GeoDataFrame(geo_list).set_crs(4326)
    
    mask = raw_gdf.geometry.apply(lambda x: isinstance(x, Polygon))
    polygon_gdf = raw_gdf[mask]
    
    filtered_gdf = polygon_gdf[polygon_gdf['name'].isin(['CADR and CALR', 'Occupied', 'Occupied Crimea'])].reset_index()
    
    merged_gdf = gpd.GeoSeries(filtered_gdf.union_all(), crs=4326)
    
    # Applying buffer to remove union artifacts
    eps = 0.000009

    deartifacted_gdf = (
        merged_gdf
        .buffer(eps, 1, join_style=JOIN_STYLE.mitre)
        .buffer(-eps, 1, join_style=JOIN_STYLE.mitre)
        )
    
    return deartifacted_gdf


In [60]:
# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Make API request
logger.info("Making API request...")
raw_data = make_api_request()

# Process data
logger.info("Processing data...")
processed_data = process_data(raw_data)

2024-09-16 15:06:39,731 - INFO - Making API request...


2024-09-16 15:06:39,957 - INFO - Processing data...


In [61]:
polygon_gdf['name'].unique()

array(['Unknown status', 'Liberated', 'Liberated 25.03',
       'Liberated 16.03-17.03', 'Liberated 23.03', 'Liberated 27-31.03',
       'Liberated 26.03', 'Liberated 27-29.03', 'Liberated 30.03',
       'Liberated 05.04', 'Liberated 31.03', 'Liberated 02.07',
       'Liberated 30.05', 'Snake Island', 'Liberated 26.06',
       'Liberated 04.09', 'Occupied Petsamo', 'Occupied Salla',
       'occupied Estonia territories.',
       'Occupied Pechorsky district (Saaste Boot).',
       'Occupied Latvia territories.', 'Occupied Southern Kuril islands.',
       'Liberated 31.03 - 02.04', 'Transnistria',
       'Occupied Tskhinvali district', 'Occupied Tuzla Island',
       'CADR and CALR', 'Occupied', 'Occupied Crimea',
       'East Prussia is temporarily occupied.', 'Occupied Abkhazia.',
       'The temporarily occupied territory of Karelia.',
       'The temporarily occupied territory of the Republic of Ichkeria.'],
      dtype=object)

In [62]:
polygon_gdf[polygon_gdf['name'].isin(['CADR and CALR', 'Occupied', 'Occupied Crimea'])].reset_index()

,index,name,geometry
0,85,CADR and CALR,"POLYGON ((39.083 47.852, 39.109 47.852, 39.11 ..."
1,86,Occupied,"POLYGON ((37.856 48.193, 37.883 48.2, 37.897 4..."
2,87,Occupied Crimea,"POLYGON ((35.201 45.523, 35.11 45.599, 35.05 4..."
3,91,Occupied,"POLYGON ((37.688 48.055, 37.691 48.053, 37.694..."
4,93,Occupied,"POLYGON ((35.253 46.249, 35.209 46.19, 35.137 ..."
5,94,Occupied,"POLYGON ((38.692 48.729, 38.754 48.718, 38.767..."
6,95,Occupied,"POLYGON ((38.114 48.766, 38.109 48.765, 38.092..."
7,96,Occupied,"POLYGON ((31.519 46.312, 31.523 46.302, 31.54 ..."
8,97,Occupied,"POLYGON ((36.401 50.307, 36.374 50.3, 36.378 5..."
9,98,Occupied,"POLYGON ((36.761 50.29, 36.755 50.291, 36.748 ..."


In [63]:
len(raw_gdf[raw_gdf.geometry.apply(lambda x: isinstance(x, Point))])

369

In [64]:
"""Create a GeoDataFrame from the processed data."""
geo_list = processed_data
raw_gdf = gpd.GeoDataFrame(geo_list).set_crs(4326)

mask = raw_gdf.geometry.apply(lambda x: isinstance(x, Polygon))
polygon_gdf = raw_gdf[mask]

filtered_gdf = polygon_gdf[polygon_gdf['name'].isin(['CADR and CALR', 'Occupied', 'Occupied Crimea'])].reset_index()

merged_gdf = gpd.GeoSeries(filtered_gdf.union_all(), crs=4326)

# Applying buffer to remove union artifacts
eps = 0.000009

deartifacted_gdf = (
    merged_gdf
    .buffer(eps, 1, join_style=JOIN_STYLE.mitre)
    .buffer(-eps, 1, join_style=JOIN_STYLE.mitre)
    )


/var/folders/dk/3hl3mmls0gs66kjcnlw3cxs80000gq/T/ipykernel_79480/2237292648.py:17: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  .buffer(eps, 1, join_style=JOIN_STYLE.mitre)
/var/folders/dk/3hl3mmls0gs66kjcnlw3cxs80000gq/T/ipykernel_79480/2237292648.py:18: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  .buffer(-eps, 1, join_style=JOIN_STYLE.mitre)


In [68]:
raw_gdf

,name,geometry
0,Unknown status,"POLYGON ((37.878 48.66, 37.885 48.652, 37.888 ..."
1,Unknown status,"POLYGON ((35.499 47.511, 35.509 47.507, 35.515..."
2,Unknown status,"POLYGON ((38.02 49.145, 38.002 49.149, 38.002 ..."
3,Unknown status,"POLYGON ((36.087 47.574, 36.106 47.579, 36.117..."
4,Unknown status,"POLYGON ((37.538 48.053, 37.517 48.074, 37.478..."
...,...,...
473,Unknown status,"POLYGON ((34.523 51.253, 34.54 51.253, 34.58 5..."
474,Liberated,"POLYGON ((34.286 51.321, 34.285 51.321, 34.283..."
475,Liberated,"POLYGON ((35.021 51.224, 35.023 51.22, 35.028 ..."
476,Liberated,"POLYGON ((35.091 51.401, 35.096 51.39, 35.142 ..."


In [67]:
# Create GeoDataFrame
logger.info("Creating GeoDataFrame...")
gdf = create_geodataframe(processed_data)

# Export as GeoJSON
output_path = os.path.join(OUTPUT_DIR, OUTPUT_FILENAME)
logger.info(f"Exporting data to {output_path}...")
gdf.to_file(output_path, driver="GeoJSON")

logger.info("Data update completed successfully.")

2024-09-16 15:06:56,076 - INFO - Creating GeoDataFrame...
/var/folders/dk/3hl3mmls0gs66kjcnlw3cxs80000gq/T/ipykernel_79480/847104444.py:94: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  .buffer(eps, 1, join_style=JOIN_STYLE.mitre)
/var/folders/dk/3hl3mmls0gs66kjcnlw3cxs80000gq/T/ipykernel_79480/847104444.py:95: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  .buffer(-eps, 1, join_style=JOIN_STYLE.mitre)
2024-09-16 15:06:56,111 - INFO - Exporting data to data/testfile_deepstatemap_data_20240916.geojson...
2024-09-16 15:06:56,119 - INFO - Created 1 records
2024-09-16 15:06:56,120 - INFO - Data update completed successfully.
